Берем результаты других ноутбуков и объединяем их тут

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ohe_noise.tabddpm_ohe_noise import *

from tqdm.notebook import tqdm

from os import listdir
from os.path import isfile, join, isdir

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

In [3]:
datename = 'default'

In [4]:
path = f'./eval/total/'
dirs = [join(path, f) for f in listdir(path) if isdir(join(path, f))]

In [5]:
results = []
for d in dirs:
    if datename in d:
        res = [pd.read_csv(join(d, f), header=[0, 1], skipinitialspace=True) for f in listdir(d) if isfile(join(d, f)) and 'csv' in f]
        results.extend(res)

results = pd.concat(results)
results = results.rename(columns={'Unnamed: 0_level_0':'', 
                                  'Unnamed: 1_level_0':'', 
                                  'Unnamed: 2_level_0':''})#.drop(columns=[('base_metrics', '0')])

In [10]:
import numpy as np
import json

with open('/workspace/TabDDPM_copy/TabDDPM/Model_names_simple.json', 'r') as openfile:
    naming = json.load(openfile)

def get_positions(s, substring):
    positions = []
    start = 0
    while True:
        start = s.find(substring, start)
        if start == -1:
            break
        positions.append(start)
        start += len(substring)
    return positions
    
def get_simple_name(model_name):
    try:
        pos = get_positions(model_name, '_')[-2]
        model_name_short = model_name[pos+1:]

        if naming.get(model_name_short):
            return naming[model_name_short]
        else:
            pos = get_positions(model_name, '_')[-3]
            model_name_short = model_name[pos+1:]
            return naming[model_name_short]
    except:
        return 'TabDDPM'

results[('', 'Model name')] = results[('', 'Type')].apply(lambda x: get_simple_name(x))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = results[[
    ('', 'Model name') ,
    ('similarity', 'Column Shapes Score, %') ,
    ('similarity', 'Column Pair Trends Score, %') ,
    ('similarity', 'Overall Score (Average), %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation, %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation std, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score std, %') ,
    ('mle', 'ROC - AUC обучения на синтетических данных') ,
    ('mle', 'ROC - AUC обучения на синтетических данных, std') ,
    ('detection', 'Score') ,
    ('DCR', 'Score') ,
    ('quality', 'alpha precision') ,
    ('quality', 'beta recall') ,
]].sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

df

similarity                              \
    Model name Column Shapes Score, % Column Pair Trends Score, %   
0   Constant 3              99.027315                   98.386669   
1   Constant 2              98.872377                   98.157852   
2   Constant 1              98.706481                   97.971314   
3     Cosine 3              98.426852                   98.106803   
4   Constant 5              98.472685                   98.038561   
5      Steps 5              98.620062                   97.679320   
6   Constant 4              98.525617                   97.665435   
7     Cosine 2              98.486728                   97.661141   
8      Steps 4              98.425617                   97.308774   
9     Linear 2              98.570833                   97.135125   
10    Cosine 1              98.621142                   96.909428   
11     Steps 3              98.360340                   97.123526   
12     TabDDPM              98.571914                   96.150060   
13    Linear 1              98.568827                   95.919682   
14     Steps 1              98.476235                   95.694245   
15     Steps 2              98.232407                   95.713741   
16  Constant 6              97.163272                   96.758654   

                               \
   Overall Score (Average), %   
0                   98.706992   
1                   98.515114   
2                   98.338898   
3                   98.266827   
4                   98.255623   
5                   98.149691   
6                   98.095526   
7                   98.073935   
8                   97.867196   
9                   97.852979   
10                  97.765285   
11                  97.741933   
12                  97.360987   
13                  97.244255   
14                  97.085240   
15                  96.973074   
16                  96.960963   

                                                        \
   Error rate (%) of column-wise density estimation, %   
0                                             0.972685   
1                                             1.127623   
2                                             1.293519   
3                                             1.573148   
4                                             1.527315   
5                                             1.379938   
6                                             1.474383   
7                                             1.513272   
8                                             1.574383   
9                                             1.429167   
10                                            1.378858   
11                                            1.639660   
12                                            1.428086   
13                                            1.431173   
14                                            1.523765   
15                                            1.767593   
16                                            2.836728   

                                                            \
   Error rate (%) of column-wise density estimation std, %   
0                                                 0.473264   
1                                                 0.453532   
2                                                 0.524780   
3                                                 0.647893   
4                                                 0.782047   
5                                                 0.602635   
6                                                 0.688450   
7                                                 0.545443   
8                                                 0.650726   
9                                                 0.601211   
10                                                0.599088   
11                                                0.673982   
12                                                0.351223   
13                                                0.55

In [13]:
df[df[('', 'Model name')].isin(['TabDDPM', 'Constant 2', 'Constant 4', 'Cosine 3'])]

similarity                              \
    Model name Column Shapes Score, % Column Pair Trends Score, %   
1   Constant 2              98.872377                   98.157852   
3     Cosine 3              98.426852                   98.106803   
6   Constant 4              98.525617                   97.665435   
12     TabDDPM              98.571914                   96.150060   

                               \
   Overall Score (Average), %   
1                   98.515114   
3                   98.266827   
6                   98.095526   
12                  97.360987   

                                                        \
   Error rate (%) of column-wise density estimation, %   
1                                             1.127623   
3                                             1.573148   
6                                             1.474383   
12                                            1.428086   

                                                            \
   Error rate (%) of column-wise density estimation std, %   
1                                                 0.453532   
3                                                 0.647893   
6                                                 0.688450   
12                                                0.351223   

                                                            \
   Error rate (%) of pair-wise column correlation score, %   
1                                                 1.842148   
3                                                 1.893197   
6                                                 2.334565   
12                                                3.849940   

                                                                \
   Error rate (%) of pair-wise column correlation score std, %   
1                                                     1.839529   
3                                                     1.051994   
6                                                     1.756375   
12                                                    5.371994   

                                          mle  \
   ROC - AUC обучения на синтетических данных   
1                                    0.720987   
3                                    0.737190   
6                                    0.725774   
12                                   0.765119   

                                                   detection       DCR  \
   ROC - AUC обучения на синтетических данных, std     Score     Score   
1                                         0.007987  0.988138  0.506963   
3                                         0.013395  0.981950  0.509000   
6                                         0.013007  0.982871  0.511222   
12                                        0.009097  0.965608  0.513852   

           quality              
   alpha precision beta recall  
1         0.984915    0.484896  
3         0.970694    0.472778  
6         0.966847    0.434185  
12        0.980190    0.477022

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

\
              Model                              Type     Data   
0   tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  default   
1   tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  default   
2   tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  default   
3   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  default   
4   tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  default   
5   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  default   
6   tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  default   
7   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  default   
8   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  default   
9   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  default   
10  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  default   
11  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  default   
12          TabDDPM                           tabddpm  default   
13  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  default   
14  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  default   
15  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  default   
16  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  default   

        base_metrics                                                  \
   Original Logistic Synthetic Logistic Original Tree Synthetic Tree   
0           0.485536           0.474687      0.491677       0.472590   
1           0.485536           0.479826      0.491677       0.443618   
2           0.485536           0.478144      0.491951       0.464646   
3           0.485536           0.484340      0.491677       0.463804   
4           0.485536           0.485479      0.491677       0.483301   
5           0.485536           0.480942      0.491677       0.449383   
6           0.485536           0.479697      0.491677       0.470297   
7           0.485536           0.479828      0.491677       0.446363   
8           0.485536           0.482529      0.491677       0.445003   
9           0.485536           0.483055      0.491677       0.433894   
10          0.485536           0.469933      0.491677       0.447682   
11          0.485536           0.465738      0.491677       0.453086   
12          0.485536           0.466703      0.492976       0.465483   
13          0.485536           0.484127      0.491677       0.496222   
14          0.485536           0.480088      0.491677       0.426081   
15          0.485536           0.478809      0.491677       0.471393   
16          0.485536           0.481122      0.491677       0.498972   

                                                               similarity  \
   Accuracy Loss Logistic, % Accuracy Loss Tree, % Column Shapes Score, %   
0                   2.234444              3.882128              99.027315   
1                   1.176123              9.774499              98.872377   
2                   1.522537              5.550267              98.706481   
3                   0.246320              5.669095              98.426852   
4                   0.011650              1.703708              98.472685   
5                   0.946235              8.602109              98.620062   
6                   1.202632              4.348443              98.525617   
7                   1.175640              9.216374              98.486728   
8                   0.619295              9.492850              98.425617   
9                   0.510902             11.752241              98.570833   
10                  3.213527              8.947986              98.621142   
11                  4.077526              7.848830              98.360340   
12                  3.878736              5.576846              98.571914   
13                  0.290202             -0.924250              98.568827   
14                  1.121961             13.341269              98.476235   
15                  1.385543              4.125531              98.232407   
16                  0.909123             -1.483607    

In [11]:
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False
                   ).reset_index(drop=True).to_excel(f'./_{datename}_notebooks/Result_united.xlsx')